In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
import matplotlib.pyplot as plt
from datetime import datetime
import warnings
import pickle
warnings.filterwarnings('ignore')

In [2]:
!unzip ../input/instacart-market-basket-analysis/order_products__prior.csv.zip 
!unzip ../input/instacart-market-basket-analysis/order_products__train.csv.zip
!unzip ../input/instacart-market-basket-analysis/orders.csv.zip
!unzip ../input/instacart-market-basket-analysis/products.csv.zip
!unzip ../input/instacart-market-basket-analysis/sample_submission.csv.zip

Archive:  ../input/instacart-market-basket-analysis/order_products__prior.csv.zip
  inflating: order_products__prior.csv  
   creating: __MACOSX/
  inflating: __MACOSX/._order_products__prior.csv  
Archive:  ../input/instacart-market-basket-analysis/order_products__train.csv.zip
  inflating: order_products__train.csv  
  inflating: __MACOSX/._order_products__train.csv  
Archive:  ../input/instacart-market-basket-analysis/orders.csv.zip
  inflating: orders.csv              
  inflating: __MACOSX/._orders.csv   
Archive:  ../input/instacart-market-basket-analysis/products.csv.zip
  inflating: products.csv            
  inflating: __MACOSX/._products.csv  
Archive:  ../input/instacart-market-basket-analysis/sample_submission.csv.zip
  inflating: sample_submission.csv   


In [2]:
prior = pd.read_csv("Data/order_products__prior.csv/order_products__prior.csv")
orders = pd.read_csv("Data/orders.csv/orders.csv", usecols=["order_id","user_id","order_number"])

## USER'S LATEST PURCHASE

In [1]:
prior_order = pd.merge(prior,orders, how="inner", on="order_id")

latest = prior_order.groupby("user_id")["order_number"].aggregate("max").reset_index()

prior_latest = pd.merge(prior_order, latest, how="inner", on=["user_id", "order_number"])

prior_latest = prior_latest[["user_id", "product_id", "reordered"]]

prior_latest.columns = ["user_id", "product_id", "usr_latest"]

NameError: name 'pd' is not defined

## USER X PRODUCTS

In [5]:
up = prior_order.groupby(["user_id","product_id"])["reordered"].aggregate(["count", "sum"]).reset_index()

up.columns = ["user_id", "product_id", "usr_prd_count", "usr_prd_sum"]

prior_order = pd.merge(up, prior_latest, how="left", on=["user_id","product_id"])

prior_order.head()

,user_id,product_id,usr_prd_count,usr_prd_sum,usr_latest
0,1,196,10,9,1.0
1,1,10258,9,8,1.0
2,1,10326,1,0,NaN
3,1,12427,10,9,1.0
4,1,13032,3,2,1.0


## USER REORDER RATE

In [22]:
## Not including this feature because it Decreasing the score of the model.

In [5]:
# prior_order['reorder_rate'] = prior_order['usr_prd_sum'] / prior_order['usr_prd_count']

# prior_order.head()

,user_id,product_id,usr_prd_count,usr_prd_sum,usr_latest,reorder_rate
0,1,196,10,9,1.0,0.900000
1,1,10258,9,8,1.0,0.888889
2,1,10326,1,0,NaN,0.000000
3,1,12427,10,9,1.0,0.900000
4,1,13032,3,2,1.0,0.666667


In [2]:
prior_order.to_pickle('prior_final.pkl')

NameError: name 'prior_order' is not defined

In [3]:
prior_order = pd.read_pickle('prior_final.pkl')

In [6]:
orders.drop(["order_number"],axis=1,inplace=True)

Train = pd.read_csv("Data/order_products__train.csv/order_products__train.csv", usecols=["order_id"])
Train = Train.groupby("order_id").aggregate("count").reset_index()

Test = pd.read_csv("Data/sample_submission.csv/sample_submission.csv", usecols=["order_id"])

Train = pd.merge(Train, orders, how="inner", on="order_id")
Test = pd.merge(Test, orders, how="inner", on="order_id")

In [7]:
tr = pd.merge(Train, prior_order, how="inner", on="user_id")
te = pd.merge(Test, prior_order, how="inner", on="user_id")

In [10]:
del prior,prior_latest,prior_order

In [8]:
te_y = pd.read_csv("Data/order_products__train.csv/order_products__train.csv", usecols=["order_id", "product_id", "reordered"])

te_y = pd.merge(te_y, orders, how="inner", on="order_id")
te_y = te_y[["user_id", "product_id", "reordered"]]


tr = pd.merge(tr, te_y, how="left", on=["user_id", "product_id"])
tr["reordered"].fillna(0, inplace=True)
del te_y

In [9]:
y = tr.reordered.values

sub_out = te[["order_id", "product_id"]]


tr_x = np.array(tr.drop(["order_id", "user_id", "reordered"], axis=1))
te_x = np.array(te.drop(["order_id", "user_id"], axis=1))

array([[1.2830e+03, 1.0000e+00, 0.0000e+00,        nan],
       [6.2910e+03, 1.0000e+00, 0.0000e+00,        nan],
       [7.0350e+03, 1.0000e+00, 0.0000e+00,        nan],
       ...,
       [4.7226e+04, 1.0000e+00, 0.0000e+00,        nan],
       [4.8790e+04, 2.0000e+00, 1.0000e+00,        nan],
       [4.8808e+04, 1.0000e+00, 0.0000e+00,        nan]])

In [14]:
start = datetime.now()
params = {}
params["objective"] = "binary:logistic"
params['eval_metric'] = 'logloss'
params["eta"] = 0.05
params["subsample"] = 0.7
params["min_child_weight"] = 10
params["colsample_bytree"] = 0.7
params["max_depth"] = 8
params["silent"] = 1

rounds = 100

par = list(params.items())

train = xgb.DMatrix(tr_x, label=y)

model = xgb.train(par, train, rounds, verbose_eval=10)

pickle.dump(model,open('xgboost.sav', 'wb'))

test = xgb.DMatrix(te_x)

prediction = model.predict(test)

print(f'Time Taken: {datetime.now()-start}')

[09:34:42] WARNING: ../src/learner.cc:516: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[09:34:43] WARNING: ../src/gbm/gbtree.cc:139: Tree method is automatically selected to be 'approx' for faster speed. To use old behavior (exact greedy algorithm on single machine), set tree_method to 'exact'.
Time Taken: 0:06:37.109964


In [ ]:
model = pickle.load(open('xgboost.sav', 'rb'))

In [17]:
cutoff = 0.2
prediction[prediction>=cutoff] = 1
prediction[prediction<cutoff] = 0
sub_out["Pred"] = prediction

In [18]:
sub_out = sub_out[sub_out['Pred']==1]

sub_out

,order_id,product_id,Pred
4,17,13107,1.0
9,17,21463,1.0
23,34,2596,1.0
40,34,13176,1.0
41,34,16083,1.0
...,...,...,...
4833242,3421054,11123,1.0
4833245,3421054,13493,1.0
4833249,3421054,15802,1.0
4833251,3421054,18426,1.0


In [19]:
def cmb_prd(x):
    return " ".join(list(x.astype('str')))

In [20]:
sub_out = sub_out.groupby("order_id")["product_id"].aggregate(cmb_prd).reset_index()
sub_out.columns = ["order_id", "products"]

In [19]:
submmission = pd.read_csv("Data/sample_submission.csv/sample_submission.csv", usecols=["order_id"])
sub_df = pd.merge(submmission, sub_out, how="left", on="order_id")


sub_df["products"].fillna("None", inplace=True)
sub_df.to_csv("submission_2.csv", index=False)

In [21]:
sub_out

,order_id,products
0,17,13107 21463
1,34,2596 13176 16083 39180 39475 44663 47029 47766...
2,137,2326 5134 23794 24852 25890 29594 41787 44422
3,182,5479 9337 13629 21903 27104 32109 33000 35951 ...
4,257,24852 27104 27966 29837 30233 35734 37646 3947...
...,...,...
70177,3420740,1158 2314 8174 11777 12016 12597 13176 13877 1...
70178,3420877,3661 5161 6567 8277 12238 13176 13646 21137 21...
70179,3420888,10880 20542 35951 43961 44632 46906
70180,3420989,5134 21616 35004 42450 43210 46676 47229 47766


In [21]:
submmission

,order_id
0,17
1,34
2,137
3,182
4,257
...,...
74995,3420740
74996,3420877
74997,3420888
74998,3420989


## Observation : 

1. I have created several features in the start in different notebooks but the score was too less.


2. I went through some notebooks and I found people are saying you don't need to create lot features. In most of the disussion most features about USER X PRODUCT, Recent Purchases.


3. I created 3 features which are Recent Purchase,USER X Product Sum and count. This features enough to put me into top 300 rank


4. I have not taken some features like time related features because of RAM limitation.


5. I can compute more features to imporve the score but the dataframe gets big and I don't enough ram to experiment.


6. I did not do any hyperparameter tuning in this notebook becuase it crashed my kernel because the data is too big for the system 


7. So I used XBoost as final model and it worked pretty well. I used XGBoost because in the first winner solution he used XGBoost as his final model.



## Reference : 


1. https://www.kaggle.com/c/instacart-market-basket-analysis/discussion/34977

2. https://www.kaggle.com/errolpereira/xgboost-with-minimal-feature-engineering#Creating-user-product-features.